In [1]:
# prepare data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os, json
import glob
import operator
from IPython.display import display, HTML

In [2]:
%load_ext autoreload
%autoreload 1

from shared_globals_functions import *
%aimport shared_globals_functions

In [3]:
t_name='t1'
merge_list=['t1.1', 't1.2', 't1.3', 't1.4']

cust_dir = 'test'

tdidf_max_df=1.0 # defaults to 1.0 (float), can be an int too
tdidf_min_df=4
use_tfidfvectorizer = True
use_stopwords = True
use_extended_stopwords = False

umap_metric='hellinger'
umap_ncomponents=2
umap_random_state=42
umap_min_dist=0 #0.1
umap_n_neighbors=30 #10, 30

min_cluster=10
max_cluster=30
clust_int=3

ss_min = 50
ss_max = 500
ss_int = 50
cs_min = 50
cs_max = 500
cs_int = 50

#tf-idf vectorizer
#max_df=0.3
#min_df=

#kmeans
#n_clusters
#n_init
#max_iter

In [4]:
# Parameters
cust_dir = "ar3/"
use_tfidfvectorizer = True
tdidf_max_df = 1.0
tdidf_min_df = 4
use_stopwords = True
use_extended_stopwords = False
umap_metric = "hellinger"
umap_ncomponents = 2
umap_random_state = 42
umap_min_dist = 0
umap_n_neighbors = 30
min_cluster = 10
max_cluster = 50
clust_int = 3
ss_min = 50
ss_max = 500
ss_int = 50
cs_min = 50
cs_max = 500
cs_int = 50
t_name = "t2.4.1s"
merge_list = ["t2.4.2", "t2.4.6"]


In [5]:
testset = from_disk(t_name)

reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t2.4.1s.pkl


In [6]:
len(testset[testset['topics'].str.len() > 0])

217922

In [7]:
if len(merge_list)>0:
    t_name = f'{t_name}+{"+".join([re.sub(r"t", "", tid) for tid in merge_list])}'
    print(t_name)

t2.4.1s+2.4.2+2.4.6


In [8]:
testset_merge = create_merge(testset, merge_list, merge_to_one=True)

reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t2.4.2.pkl


reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/t2.4.6.pkl


In [9]:
len(testset_merge[testset_merge['topics'].str.len() > 0])

217922

In [10]:
df = testset_merge.merge(testset, on=['name'], how='left', indicator=True)

In [11]:
df[df['_merge']!='both']

,name,topics_x,topics_y,_merge


In [12]:
df[df['topics_x'].str.len() == 0]

,name,topics_x,topics_y,_merge


In [13]:
if len(merge_list)>0:
    testset = testset_merge

In [14]:
import pandas as pd
import umap
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [15]:
orig_testset = testset

In [16]:
my_stop_words=None
if use_stopwords:
    my_stop_words='english'
    if use_extended_stopwords:
        print("todo")

if use_tfidfvectorizer:
    vectorizer = TfidfVectorizer(min_df=tdidf_min_df, max_df=tdidf_max_df, stop_words=my_stop_words)
else:
    vectorizer = CountVectorizer(min_df=tdidf_min_df, max_df=tdidf_max_df, stop_words=my_stop_words)

print(f"total repos {len(testset[testset['topics'].str.len() > 0])}")
testset = testset[testset['topics'].str.len() > 0].reset_index()

if isinstance(testset['topics'][0], list):
    testset['topics'] = testset['topics'].apply(lambda x: ' '.join(x))
    print("topics came as list ... consider joining it before")

cr_start()
word_doc_matrix = vectorizer.fit_transform(testset['topics'])
runtime_vectorizer = cr_getrt()

total repos 217922


topics came as list ... consider joining it before


In [17]:
runtime_vectorizer

3.410484790802002

In [18]:
word_doc_matrix

<217922x728 sparse matrix of type '<class 'numpy.float64'>'
	with 6503510 stored elements in Compressed Sparse Row format>

In [19]:
class ProgressWriter:
    def write(self, text):
        print("custom progress", n, total)

    def flush(self):
        pass

#tqdm_kwds = {"file": "progress_writer", "disable": False }
tqdm_kwds = {"disable": False }   


In [20]:
%%time
# try also with default parameters for n_neighbors and min_dist

cr_start()

embedding = umap.UMAP(
    n_neighbors=umap_n_neighbors,
    min_dist=umap_min_dist,
    n_components=umap_ncomponents, metric=umap_metric,
    tqdm_kwds=tqdm_kwds,
    random_state=umap_random_state,
).fit(word_doc_matrix)

runtime_umap = cr_getrt()

/home/manuel/d3tect-tools/venv/lib/python3.9/site-packages/umap/umap_.py:125: UserWarning: A few of your vertices were disconnected from the manifold.  This shouldn't cause problems.
Disconnection_distance = 1 has removed 646 edges.
It has only fully disconnected 16 vertices.
Use umap.utils.disconnected_vertices() to identify them.
  warn(


Epochs completed:   0%|            0/200 [00:00]

CPU times: user 38min 31s, sys: 1min 31s, total: 40min 3s
Wall time: 9min 4s


In [21]:
print(runtime_umap)

544.4622259140015


In [22]:
%%time

cr_start()
clusterable_embedding = embedding.transform(word_doc_matrix)
runtime_umap_transform = cr_getrt()

/home/manuel/d3tect-tools/venv/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


Epochs completed:   0%|            0/30 [00:00]

CPU times: user 24min 35s, sys: 1.4 s, total: 24min 37s
Wall time: 23min 1s


In [23]:
print(runtime_umap_transform)

1381.1422913074493


In [24]:
umap_embedding = pd.DataFrame(embedding.embedding_, columns=['x', 'y']) # (umap_t)
umap_clusterable_embedding = pd.DataFrame(clusterable_embedding, columns=['x', 'y']) # (umap_t)

umap_embedding['name'] = testset['name']
umap_clusterable_embedding['name'] = testset['name']

to_disk_results(umap_embedding, f'umap_embedding_{t_name}', subdir=f"{cust_dir}")
to_disk_results(umap_clusterable_embedding, f'umap_clusterable_embedding_{t_name}', subdir=f"{cust_dir}")

writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//umap_embedding_t2.4.1s+2.4.2+2.4.6.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//umap_clusterable_embedding_t2.4.1s+2.4.2+2.4.6.pkl


https://hdbscan.readthedocs.io/en/latest/parameter_selection.html

Since we have seen that min_samples clearly has a dramatic effect on clustering, the question becomes: how do we select this parameter? The simplest intuition for what min_samples does is provide a measure of how conservative you want you clustering to be. The larger the value of min_samples you provide, the more conservative the clustering – more points will be declared as noise, and clusters will be restricted to progressively more dense areas. We can see this in practice by leaving the min_cluster_size at 60, but reducing min_samples to 1.

In [25]:
import hdbscan

def clusterapply_hdbscan(hdbscan_min_samples, min_cluster_size, runner_name):
    cr_start()
    labels = hdbscan.HDBSCAN(
        min_samples=hdbscan_min_samples,
        min_cluster_size=min_cluster_size,
    ).fit_predict(clusterable_embedding)
    runtime_hdb = cr_getrt()
    
    stats = {'type': "hdb",
             'testset': t_name,
             'version': td_version,
            }
    
    detail_stats = {'hdb_minclust': min_cluster_size,
                    'hdb_minsamp': hdbscan_min_samples,
                    'runtime': runtime_hdb,
                    'tdf': f"rt_vect: {runtime_vectorizer}, min_df: {tdidf_min_df}, max_df: {tdidf_max_df}, stopwords: {use_stopwords}, extended stopwords: {use_extended_stopwords}",
                    'umap': f"{umap_metric}, rt_umap_tf: {runtime_umap_transform}, rt_umap: {runtime_umap}, ncomp: {umap_ncomponents}, rs: {umap_random_state}, min_dist: {umap_min_dist}, no_neigh: {umap_n_neighbors}",
                    #'gtr': grtr,
                    #'gtr_rel': grtr_rt
                   }
    prediction = stats
    prediction['name'] = runner_name
    #prediction.update(max_fscore(my_results[[tag, 'clabel']].apply(tuple, axis=1).to_dict()))
    #prediction.update({'tot_clust': total_clusters, 'tot_anom':no_anomalies, 'no_lab':missing_clabel, 'anom':identified_anom})
    prediction.update(detail_stats)
    res.append(prediction.copy())
    display(pd.DataFrame.from_dict(res))
    return labels

res = []
hdb_labels_results = pd.DataFrame()
hdb_labels_results['name'] = testset['name'].copy()


for cs in range(cs_min, cs_max, cs_int): #50, 500
    for ss in range(ss_min, ss_max, ss_int): #50 500
        runner_name = f'ss{ss}cs{cs}'
        hdb_labels_results[runner_name] = clusterapply_hdbscan(ss, cs, runner_name)

results = pd.DataFrame.from_dict(res)
res = []


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.1s+2.4.2+2.4.6,1,ss50cs50,50,50,5.437215,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.1s+2.4.2+2.4.6,1,ss50cs50,50,50,5.437215,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
1,hdb,t2.4.1s+2.4.2+2.4.6,1,ss100cs50,50,100,5.403534,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.1s+2.4.2+2.4.6,1,ss50cs50,50,50,5.437215,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
1,hdb,t2.4.1s+2.4.2+2.4.6,1,ss100cs50,50,100,5.403534,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
2,hdb,t2.4.1s+2.4.2+2.4.6,1,ss150cs50,50,150,6.127807,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.1s+2.4.2+2.4.6,1,ss50cs50,50,50,5.437215,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
1,hdb,t2.4.1s+2.4.2+2.4.6,1,ss100cs50,50,100,5.403534,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
2,hdb,t2.4.1s+2.4.2+2.4.6,1,ss150cs50,50,150,6.127807,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
3,hdb,t2.4.1s+2.4.2+2.4.6,1,ss200cs50,50,200,6.819609,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.1s+2.4.2+2.4.6,1,ss50cs50,50,50,5.437215,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
1,hdb,t2.4.1s+2.4.2+2.4.6,1,ss100cs50,50,100,5.403534,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
2,hdb,t2.4.1s+2.4.2+2.4.6,1,ss150cs50,50,150,6.127807,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
3,hdb,t2.4.1s+2.4.2+2.4.6,1,ss200cs50,50,200,6.819609,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
4,hdb,t2.4.1s+2.4.2+2.4.6,1,ss250cs50,50,250,7.837960,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."


...

,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.1s+2.4.2+2.4.6,1,ss50cs50,50,50,5.437215,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
1,hdb,t2.4.1s+2.4.2+2.4.6,1,ss100cs50,50,100,5.403534,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
2,hdb,t2.4.1s+2.4.2+2.4.6,1,ss150cs50,50,150,6.127807,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
3,hdb,t2.4.1s+2.4.2+2.4.6,1,ss200cs50,50,200,6.819609,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
4,hdb,t2.4.1s+2.4.2+2.4.6,1,ss250cs50,50,250,7.837960,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
...,...,...,...,...,...,...,...,...,...
72,hdb,t2.4.1s+2.4.2+2.4.6,1,ss50cs450,450,50,4.919142,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
73,hdb,t2.4.1s+2.4.2+2.4.6,1,ss100cs450,450,100,5.410082,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
74,hdb,t2.4.1s+2.4.2+2.4.6,1,ss150cs450,450,150,6.064618,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
75,hdb,t2.4.1s+2.4.2+2.4.6,1,ss200cs450,450,200,7.301127,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.1s+2.4.2+2.4.6,1,ss50cs50,50,50,5.437215,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
1,hdb,t2.4.1s+2.4.2+2.4.6,1,ss100cs50,50,100,5.403534,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
2,hdb,t2.4.1s+2.4.2+2.4.6,1,ss150cs50,50,150,6.127807,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
3,hdb,t2.4.1s+2.4.2+2.4.6,1,ss200cs50,50,200,6.819609,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
4,hdb,t2.4.1s+2.4.2+2.4.6,1,ss250cs50,50,250,7.837960,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
...,...,...,...,...,...,...,...,...,...
73,hdb,t2.4.1s+2.4.2+2.4.6,1,ss100cs450,450,100,5.410082,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
74,hdb,t2.4.1s+2.4.2+2.4.6,1,ss150cs450,450,150,6.064618,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
75,hdb,t2.4.1s+2.4.2+2.4.6,1,ss200cs450,450,200,7.301127,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
76,hdb,t2.4.1s+2.4.2+2.4.6,1,ss250cs450,450,250,7.590310,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.1s+2.4.2+2.4.6,1,ss50cs50,50,50,5.437215,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
1,hdb,t2.4.1s+2.4.2+2.4.6,1,ss100cs50,50,100,5.403534,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
2,hdb,t2.4.1s+2.4.2+2.4.6,1,ss150cs50,50,150,6.127807,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
3,hdb,t2.4.1s+2.4.2+2.4.6,1,ss200cs50,50,200,6.819609,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
4,hdb,t2.4.1s+2.4.2+2.4.6,1,ss250cs50,50,250,7.837960,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
...,...,...,...,...,...,...,...,...,...
74,hdb,t2.4.1s+2.4.2+2.4.6,1,ss150cs450,450,150,6.064618,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
75,hdb,t2.4.1s+2.4.2+2.4.6,1,ss200cs450,450,200,7.301127,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
76,hdb,t2.4.1s+2.4.2+2.4.6,1,ss250cs450,450,250,7.590310,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
77,hdb,t2.4.1s+2.4.2+2.4.6,1,ss300cs450,450,300,8.316549,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.1s+2.4.2+2.4.6,1,ss50cs50,50,50,5.437215,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
1,hdb,t2.4.1s+2.4.2+2.4.6,1,ss100cs50,50,100,5.403534,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
2,hdb,t2.4.1s+2.4.2+2.4.6,1,ss150cs50,50,150,6.127807,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
3,hdb,t2.4.1s+2.4.2+2.4.6,1,ss200cs50,50,200,6.819609,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
4,hdb,t2.4.1s+2.4.2+2.4.6,1,ss250cs50,50,250,7.837960,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
...,...,...,...,...,...,...,...,...,...
75,hdb,t2.4.1s+2.4.2+2.4.6,1,ss200cs450,450,200,7.301127,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
76,hdb,t2.4.1s+2.4.2+2.4.6,1,ss250cs450,450,250,7.590310,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
77,hdb,t2.4.1s+2.4.2+2.4.6,1,ss300cs450,450,300,8.316549,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
78,hdb,t2.4.1s+2.4.2+2.4.6,1,ss350cs450,450,350,9.007425,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."


,type,testset,version,name,hdb_minclust,hdb_minsamp,runtime,tdf,umap
0,hdb,t2.4.1s+2.4.2+2.4.6,1,ss50cs50,50,50,5.437215,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
1,hdb,t2.4.1s+2.4.2+2.4.6,1,ss100cs50,50,100,5.403534,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
2,hdb,t2.4.1s+2.4.2+2.4.6,1,ss150cs50,50,150,6.127807,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
3,hdb,t2.4.1s+2.4.2+2.4.6,1,ss200cs50,50,200,6.819609,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
4,hdb,t2.4.1s+2.4.2+2.4.6,1,ss250cs50,50,250,7.837960,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
...,...,...,...,...,...,...,...,...,...
76,hdb,t2.4.1s+2.4.2+2.4.6,1,ss250cs450,450,250,7.590310,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
77,hdb,t2.4.1s+2.4.2+2.4.6,1,ss300cs450,450,300,8.316549,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
78,hdb,t2.4.1s+2.4.2+2.4.6,1,ss350cs450,450,350,9.007425,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."
79,hdb,t2.4.1s+2.4.2+2.4.6,1,ss400cs450,450,400,9.482081,"rt_vect: 3.410484790802002, min_df: 4, max_df:...","hellinger, rt_umap_tf: 1381.1422913074493, rt_..."


In [26]:
hdb_labels_results

,name,ss50cs50,ss100cs50,ss150cs50,ss200cs50,ss250cs50,ss300cs50,ss350cs50,ss400cs50,ss450cs50,...,ss450cs400,ss50cs450,ss100cs450,ss150cs450,ss200cs450,ss250cs450,ss300cs450,ss350cs450,ss400cs450,ss450cs450
0,technopagan/mss-saliency,-1,-1,-1,3,4,5,4,4,4,...,3,-1,-1,16,3,3,3,3,3,3
1,snmnmin12/selective-search,-1,-1,-1,3,4,5,4,4,4,...,3,-1,-1,16,3,3,3,3,3,3
2,google/chaps-linux,-1,-1,-1,3,4,5,4,4,4,...,3,-1,-1,16,3,3,3,3,3,3
3,lukasolson/speech-router,205,58,12,3,4,5,4,4,4,...,3,36,18,7,3,3,3,3,3,3
4,XavierTecBr/qbr-admin,-1,-1,-1,3,4,5,4,4,4,...,3,-1,-1,-1,3,3,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217917,bhagatabhijeet/bhagatabhijeet,-1,-1,-1,3,4,5,4,4,4,...,3,-1,-1,-1,3,3,3,3,3,3
217918,LandGrey/SpringBootVulExploit,110,54,-1,3,4,5,4,4,4,...,3,-1,-1,16,3,3,3,3,3,3
217919,mucomplex/Pentest_checklist,131,60,38,3,4,5,4,4,4,...,3,19,20,16,3,3,3,3,3,3
217920,bigsen/Vision_Track,175,80,42,3,4,5,4,4,4,...,3,37,32,16,3,3,3,3,3,3


In [27]:
print("saving results to disk")
try:
    old_results = from_disk(f'results/{cust_dir}/hdb_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'hdb_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'hdb_results', subdir=f"{cust_dir}")
to_disk_results(hdb_labels_results, f'hdb_labels_results_{t_name}', subdir=f"{cust_dir}")

saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//hdb_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//hdb_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//hdb_labels_results_t2.4.1s+2.4.2+2.4.6.pkl


In [28]:

umap_embedding['topics'] = testset['topics']
umap_clusterable_embedding['topics'] = testset['topics']

In [29]:
import sklearn.cluster as cluster

def clusterapply_kmeans(testset, clustersize, runner_name, opt_comment=""):
    # Perform k-means clustering
    
    kmeans = cluster.KMeans(n_clusters=clustersize, random_state=42)
        
    if('y' in testset.columns):
        #print("assuming umap reduced testset here")
        testset = testset.copy()
        dropping_not_connected = len(testset[testset['y'].isnull()])
        testset = testset[testset['y'].notna()]
        cr_start()
        kmeans.fit(testset[['x', 'y']].to_numpy())
        runtime_kmeans = cr_getrt()
        opt_comment="-umapt"
    else:
        dropping_not_connected = 0
        cr_start()
        kmeans.fit(word_doc_matrix)
        runtime_kmeans = cr_getrt()

    
    stats = {'type': f"kmeans{opt_comment}",
             'testset': t_name,
             'version': td_version,
             'size': clustersize
            }
    
    detail_stats = {'tdf': f"rt_vect: {runtime_vectorizer}, min_df: {tdidf_min_df}, max_df: {tdidf_max_df}, stopwords: {use_stopwords}, extended stopwords: {use_extended_stopwords}",
                'runtime': runtime_kmeans
               }
    
    if('y' in testset.columns):
        detail_stats['umap'] = f"{umap_metric}, rt_umap_tf: {runtime_umap_transform}, rt_umap: {runtime_umap}, ncomp: {umap_ncomponents}, rs: {umap_random_state}, min_dist: {umap_min_dist}, no_neigh: {umap_n_neighbors}", #umap not important here
        stats['no_con'] = dropping_not_connected

    prediction = stats
    prediction['name'] = runner_name
    #prediction.update(max_fscore(my_results[[tag, 'clabel']].apply(tuple, axis=1).to_dict()))
    #prediction.update({'tot_clust': total_clusters, 'no_lab':missing_clabel})
    prediction.update(detail_stats)
    res.append(prediction.copy())
        
    display(pd.DataFrame.from_dict(res))
    return kmeans.labels_
    #testset.drop(['cluster_run'], axis=1)


In [30]:
res = []

kmeans_labels_results = pd.DataFrame()
kmeans_labels_results['name'] = umap_embedding[umap_embedding['y'].notna()]['name'].copy()

for i in range(min_cluster, max_cluster, clust_int): # 10, 60
    runner_name = f'{i}'
    kmeans_labels_results[runner_name] = clusterapply_kmeans(umap_embedding, i, runner_name)

results = pd.DataFrame.from_dict(res)
res = []

#k_means_df[['x', 'y']].to_numpy(), 
#word_doc_matrix



,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,10,16,10,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",2.141496,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,10,16,10,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",2.141496,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
1,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,13,16,13,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",2.997792,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,10,16,10,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",2.141496,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
1,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,13,16,13,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",2.997792,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
2,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,16,16,16,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",2.968656,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,10,16,10,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",2.141496,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
1,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,13,16,13,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",2.997792,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
2,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,16,16,16,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",2.968656,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
3,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,19,16,19,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",3.593562,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,10,16,10,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",2.141496,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
1,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,13,16,13,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",2.997792,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
2,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,16,16,16,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",2.968656,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
3,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,19,16,19,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",3.593562,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
4,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,22,16,22,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",4.092786,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."


...

,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,10,16,10,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",2.141496,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
1,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,13,16,13,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",2.997792,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
2,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,16,16,16,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",2.968656,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
3,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,19,16,19,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",3.593562,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
4,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,22,16,22,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",4.092786,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
5,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,25,16,25,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",4.773751,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
6,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,28,16,28,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",5.189238,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
7,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,31,16,31,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",5.280701,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
8,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,34,16,34,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",5.877689,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
9,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,37,16,37,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",5.410515,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,10,16,10,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",2.141496,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
1,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,13,16,13,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",2.997792,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
2,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,16,16,16,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",2.968656,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
3,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,19,16,19,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",3.593562,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
4,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,22,16,22,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",4.092786,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
5,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,25,16,25,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",4.773751,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
6,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,28,16,28,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",5.189238,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
7,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,31,16,31,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",5.280701,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
8,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,34,16,34,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",5.877689,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
9,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,37,16,37,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",5.410515,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,10,16,10,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",2.141496,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
1,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,13,16,13,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",2.997792,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
2,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,16,16,16,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",2.968656,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
3,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,19,16,19,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",3.593562,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
4,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,22,16,22,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",4.092786,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
5,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,25,16,25,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",4.773751,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
6,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,28,16,28,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",5.189238,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
7,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,31,16,31,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",5.280701,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
8,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,34,16,34,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",5.877689,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
9,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,37,16,37,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",5.410515,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,10,16,10,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",2.141496,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
1,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,13,16,13,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",2.997792,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
2,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,16,16,16,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",2.968656,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
3,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,19,16,19,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",3.593562,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
4,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,22,16,22,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",4.092786,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
5,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,25,16,25,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",4.773751,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
6,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,28,16,28,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",5.189238,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
7,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,31,16,31,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",5.280701,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
8,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,34,16,34,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",5.877689,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
9,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,37,16,37,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",5.410515,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."


,type,testset,version,size,no_con,name,tdf,runtime,umap
0,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,10,16,10,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",2.141496,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
1,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,13,16,13,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",2.997792,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
2,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,16,16,16,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",2.968656,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
3,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,19,16,19,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",3.593562,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
4,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,22,16,22,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",4.092786,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
5,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,25,16,25,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",4.773751,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
6,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,28,16,28,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",5.189238,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
7,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,31,16,31,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",5.280701,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
8,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,34,16,34,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",5.877689,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."
9,kmeans-umapt,t2.4.1s+2.4.2+2.4.6,1,37,16,37,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",5.410515,"(hellinger, rt_umap_tf: 1381.1422913074493, rt..."


In [31]:
kmeans_labels_results

,name,10,13,16,19,22,25,28,31,34,37,40,43,46,49
0,technopagan/mss-saliency,9,0,11,1,1,15,11,1,1,18,24,16,36,18
1,snmnmin12/selective-search,9,0,11,1,1,15,11,1,1,18,2,16,1,43
2,google/chaps-linux,8,10,3,18,19,6,0,11,23,31,13,14,44,29
3,lukasolson/speech-router,4,1,10,14,0,3,19,24,17,23,18,32,22,0
4,XavierTecBr/qbr-admin,8,7,2,2,21,6,12,11,31,16,3,1,19,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217917,bhagatabhijeet/bhagatabhijeet,8,7,2,2,21,24,12,14,14,25,34,1,21,23
217918,LandGrey/SpringBootVulExploit,2,5,8,7,8,19,23,18,16,15,32,28,8,2
217919,mucomplex/Pentest_checklist,8,2,6,6,9,0,8,16,31,16,36,20,33,3
217920,bigsen/Vision_Track,9,0,11,1,1,15,11,1,1,18,24,16,36,18


In [32]:
print("saving results to disk")
try:
    old_results = from_disk(f'results/{cust_dir}/kmeans-umap_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'kmeans-umap_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'kmeans-umap_results', subdir=f"{cust_dir}")
    
to_disk_results(kmeans_labels_results, f'kmeans-umap_labels_results_{t_name}', subdir=f"{cust_dir}")


saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans-umap_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans-umap_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans-umap_labels_results_t2.4.1s+2.4.2+2.4.6.pkl


In [33]:
res = []

kmeans_labels_results = pd.DataFrame()
kmeans_labels_results['name'] = testset['name'].copy()


for i in range(min_cluster, max_cluster, clust_int): # 10, 60
    runner_name = f'{i}'
    kmeans_labels_results[runner_name] = clusterapply_kmeans(testset, i, runner_name)

results = pd.DataFrame.from_dict(res)
res = []

#k_means_df[['x', 'y']].to_numpy(), 
#word_doc_matrix

,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.1s+2.4.2+2.4.6,1,10,10,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",19.783211


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.1s+2.4.2+2.4.6,1,10,10,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",19.783211
1,kmeans,t2.4.1s+2.4.2+2.4.6,1,13,13,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",30.650607


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.1s+2.4.2+2.4.6,1,10,10,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",19.783211
1,kmeans,t2.4.1s+2.4.2+2.4.6,1,13,13,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",30.650607
2,kmeans,t2.4.1s+2.4.2+2.4.6,1,16,16,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",31.012016


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.1s+2.4.2+2.4.6,1,10,10,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",19.783211
1,kmeans,t2.4.1s+2.4.2+2.4.6,1,13,13,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",30.650607
2,kmeans,t2.4.1s+2.4.2+2.4.6,1,16,16,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",31.012016
3,kmeans,t2.4.1s+2.4.2+2.4.6,1,19,19,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",41.759168


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.1s+2.4.2+2.4.6,1,10,10,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",19.783211
1,kmeans,t2.4.1s+2.4.2+2.4.6,1,13,13,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",30.650607
2,kmeans,t2.4.1s+2.4.2+2.4.6,1,16,16,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",31.012016
3,kmeans,t2.4.1s+2.4.2+2.4.6,1,19,19,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",41.759168
4,kmeans,t2.4.1s+2.4.2+2.4.6,1,22,22,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",50.465735


...

,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.1s+2.4.2+2.4.6,1,10,10,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",19.783211
1,kmeans,t2.4.1s+2.4.2+2.4.6,1,13,13,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",30.650607
2,kmeans,t2.4.1s+2.4.2+2.4.6,1,16,16,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",31.012016
3,kmeans,t2.4.1s+2.4.2+2.4.6,1,19,19,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",41.759168
4,kmeans,t2.4.1s+2.4.2+2.4.6,1,22,22,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",50.465735
5,kmeans,t2.4.1s+2.4.2+2.4.6,1,25,25,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",60.344163
6,kmeans,t2.4.1s+2.4.2+2.4.6,1,28,28,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",59.452343
7,kmeans,t2.4.1s+2.4.2+2.4.6,1,31,31,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",66.383692
8,kmeans,t2.4.1s+2.4.2+2.4.6,1,34,34,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",90.204854
9,kmeans,t2.4.1s+2.4.2+2.4.6,1,37,37,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",89.536202


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.1s+2.4.2+2.4.6,1,10,10,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",19.783211
1,kmeans,t2.4.1s+2.4.2+2.4.6,1,13,13,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",30.650607
2,kmeans,t2.4.1s+2.4.2+2.4.6,1,16,16,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",31.012016
3,kmeans,t2.4.1s+2.4.2+2.4.6,1,19,19,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",41.759168
4,kmeans,t2.4.1s+2.4.2+2.4.6,1,22,22,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",50.465735
5,kmeans,t2.4.1s+2.4.2+2.4.6,1,25,25,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",60.344163
6,kmeans,t2.4.1s+2.4.2+2.4.6,1,28,28,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",59.452343
7,kmeans,t2.4.1s+2.4.2+2.4.6,1,31,31,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",66.383692
8,kmeans,t2.4.1s+2.4.2+2.4.6,1,34,34,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",90.204854
9,kmeans,t2.4.1s+2.4.2+2.4.6,1,37,37,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",89.536202


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.1s+2.4.2+2.4.6,1,10,10,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",19.783211
1,kmeans,t2.4.1s+2.4.2+2.4.6,1,13,13,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",30.650607
2,kmeans,t2.4.1s+2.4.2+2.4.6,1,16,16,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",31.012016
3,kmeans,t2.4.1s+2.4.2+2.4.6,1,19,19,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",41.759168
4,kmeans,t2.4.1s+2.4.2+2.4.6,1,22,22,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",50.465735
5,kmeans,t2.4.1s+2.4.2+2.4.6,1,25,25,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",60.344163
6,kmeans,t2.4.1s+2.4.2+2.4.6,1,28,28,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",59.452343
7,kmeans,t2.4.1s+2.4.2+2.4.6,1,31,31,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",66.383692
8,kmeans,t2.4.1s+2.4.2+2.4.6,1,34,34,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",90.204854
9,kmeans,t2.4.1s+2.4.2+2.4.6,1,37,37,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",89.536202


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.1s+2.4.2+2.4.6,1,10,10,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",19.783211
1,kmeans,t2.4.1s+2.4.2+2.4.6,1,13,13,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",30.650607
2,kmeans,t2.4.1s+2.4.2+2.4.6,1,16,16,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",31.012016
3,kmeans,t2.4.1s+2.4.2+2.4.6,1,19,19,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",41.759168
4,kmeans,t2.4.1s+2.4.2+2.4.6,1,22,22,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",50.465735
5,kmeans,t2.4.1s+2.4.2+2.4.6,1,25,25,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",60.344163
6,kmeans,t2.4.1s+2.4.2+2.4.6,1,28,28,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",59.452343
7,kmeans,t2.4.1s+2.4.2+2.4.6,1,31,31,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",66.383692
8,kmeans,t2.4.1s+2.4.2+2.4.6,1,34,34,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",90.204854
9,kmeans,t2.4.1s+2.4.2+2.4.6,1,37,37,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",89.536202


,type,testset,version,size,name,tdf,runtime
0,kmeans,t2.4.1s+2.4.2+2.4.6,1,10,10,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",19.783211
1,kmeans,t2.4.1s+2.4.2+2.4.6,1,13,13,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",30.650607
2,kmeans,t2.4.1s+2.4.2+2.4.6,1,16,16,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",31.012016
3,kmeans,t2.4.1s+2.4.2+2.4.6,1,19,19,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",41.759168
4,kmeans,t2.4.1s+2.4.2+2.4.6,1,22,22,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",50.465735
5,kmeans,t2.4.1s+2.4.2+2.4.6,1,25,25,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",60.344163
6,kmeans,t2.4.1s+2.4.2+2.4.6,1,28,28,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",59.452343
7,kmeans,t2.4.1s+2.4.2+2.4.6,1,31,31,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",66.383692
8,kmeans,t2.4.1s+2.4.2+2.4.6,1,34,34,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",90.204854
9,kmeans,t2.4.1s+2.4.2+2.4.6,1,37,37,"rt_vect: 3.410484790802002, min_df: 4, max_df:...",89.536202


In [34]:
kmeans_labels_results

,name,10,13,16,19,22,25,28,31,34,37,40,43,46,49
0,technopagan/mss-saliency,7,2,6,2,19,13,26,27,13,27,11,3,24,5
1,snmnmin12/selective-search,7,2,6,2,19,13,26,27,13,27,11,3,24,4
2,google/chaps-linux,0,11,2,13,21,10,7,1,7,7,10,1,35,18
3,lukasolson/speech-router,4,8,14,0,16,18,4,29,9,4,38,2,38,48
4,XavierTecBr/qbr-admin,9,12,2,14,20,23,12,1,0,21,18,10,33,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217917,bhagatabhijeet/bhagatabhijeet,5,1,5,18,9,6,18,18,23,28,37,37,22,21
217918,LandGrey/SpringBootVulExploit,9,12,2,14,20,23,12,1,28,21,33,10,33,3
217919,mucomplex/Pentest_checklist,0,0,11,15,2,22,1,25,16,7,5,11,32,13
217920,bigsen/Vision_Track,1,5,7,2,6,2,17,2,10,23,32,41,2,0


In [35]:
print("saving results to disk")

try:
    old_results = from_disk(f'results/{cust_dir}/kmeans_results')
    concat_results = pd.concat([old_results, results])
    concat_results.drop_duplicates(inplace=True)
    to_disk_results(concat_results, f'kmeans_results', subdir=f"{cust_dir}")
except FileNotFoundError:
    to_disk_results(results, f'kmeans_results', subdir=f"{cust_dir}")
    
to_disk_results(kmeans_labels_results, f'kmeans_labels_results_{t_name}', subdir=f"{cust_dir}")


saving results to disk
reading /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans_results.pkl
writing /home/manuel/d3tect-tools/Notebooks/raw_testdata_v1/results/ar3//kmeans_labels_results_t2.4.1s+2.4.2+2.4.6.pkl
